University of Michigan - ROB 101 Computational Linear Algebra

# Lab 7. Grahm-Schmidt Algorithm Continued, Subspaces, and Solutions to Ax=b with A a Tall Matrix! 


### Problem 1.  Build a Grahm-Schmidt function that produces an orthonormal basis
Hint: Check out the General step shown in the Grahm-Schmidt algorithm taken from the ROB 101 booklet.  Think about how you can use code to repeat that step in order to find the orthogonal basis.  Remember to normalize as we did above!

<img src = "https://i.postimg.cc/Wz9BtyRb/grahm-schmidt.png" width = 700>

For simplicity, your function will take the vectors stacked as the columns of a matrix U and output the orthonormal vectors as the columns of a matrix V, just as we did above.

In [ ]:
using LinearAlgebra

#Build your function here! We will give you a shell that you are to complete
function grahm_schmidt(U)
    # We assume the columns of U are linearly independent. Why?
    # Because, with this assumption, norm(vk) will never be zero
    # start a blank matrix V which will hold all of the output vectors
    (rows, cols) = size(U)
    V = Array{Float64,2}(undef, rows, 0)
    # Start a for loop that runs the number of times that there are columns in U
    for k in 1:cols
        # Insert the general step of Gram-Schmidt here. You will need another for loop to evaluate the summation
        # your code here
        throw(ErrorException())
        V = [V vk/norm(vk)] # We do the normalization for you!
    end
    return V
end     

In [ ]:
# Friendly self test
using Random
Random.seed!(2021)
U = rand(12, 12)
solTest = grahm_schmidt(U)
T1 = @assert(norm(solTest'*solTest - I) < 1e-10 )
println("all nothings means likely correct")
[T1]

In [ ]:
# Friendly self test
Random.seed!(1999)
Ubig = rand(100, 27)
sol2a = grahm_schmidt(Ubig)
T1=@assert isapprox(sol2a[1, 1], 0.0834795, atol = 1E-6)
T2=@assert isapprox(sol2a[100, 27], 0.0523471, atol = 1E-6)
println("all nothings means likely correct")
[T1 T2]

### Problem 2. Finding the Basis

#### a) Gram-Schmidt with linearly dependent vectors 

Use your Gram-Schmidt code from above (that is, copy it!) to complete this version of Gram-Schmidt that removes the requirement that the vectors be linearly indepenent. We will then apply our new-and-improved function to the problem of existence and uniqueness.


In [ ]:

#Build your function here! We give you a shell that we all complete together
function grahm_schmidt2(U)
    # start a blank matrix V which will hold all of the output vectors
    (rows, cols) = size(U)
    V = Array{Float64,2}(undef, rows, 0)
    aTol=1e-8
    #start a for loop that runs the number of times that there are columns in U
    for k in 1:cols
        #insert the general step of Gram-Schmidt here from your work in part a)
        # your code here
        throw(ErrorException())
        if norm(vk) > aTol  # Trick? Not really. We only add a normalized vk 
           V = [V vk/norm(vk)]   # if vk is not approximately the zero vector. Make sense?
        end  # And we ignore the vector if it is dependent on the previous vectors
    end
    return V
end

#### b) Find a set of linearly independet vectors that span the columns of a wide matrix (more columns than rows). In Chapter 10, we will call such vectors a basis.

There are many solutions to this, but we'll use Gram-Schmidt and build and orthonormal set of vectors because they are guaranteed to be linearly independent, AND we know that GS is span preserving (after we discard the dependent vectors)!

In [ ]:
# Run me don't change me
A=randn(5, 50)

# your code here
throw(ErrorException())

In [ ]:
# Run me don't change me
# Check that V'*A expresses each column of A as a linear combination of the columns of V 

Acol13=A[:,13]
@show alpha=V'*Acol13
@show norm(Acol13-V*alpha)

Observe that $V * alpha = v1 * alpha1 + v2 *alpha2 + .... + vm * alpham$, where vi are the columns of V.

### Grahm-Schmidt for determining existence of solutions.

We know that a solution to $Ax = b$ exists **IF AND ONLY IF** $b \in \operatorname {col~span} \{A\} $. Once we have a Gram-Schmidt algorithm that removes dependent vectors, then we can say that
$b \in \operatorname {col~span} \{A\} $ **IF AND ONLY IF** $ \operatorname{grahm-schmidt}(A)$ and $\operatorname {grahm-schmidt}([A, b]) $ return the same **number of orthonormal vectors**, because if b were **linearly independent** of the columns of A, then [A b] would have one more linearly independent vector than A!

### Problem 3.  Checking for existence of solutions to $Ax = b$ 
Use your grahm-schmidt function to check for existence of solutions. The return statement is completed for you. 

In [ ]:
#use your grahm_schmidt function inside this function
#Assume that A and b are size compatible
function sol_exist(A, b)
    #fill in the missing part of the function. You must let 
    # nColsGS_A be the number of orthonormal vectors returned by grahm_schmidt applied to the matrix A
    # and let nColsGS_Ab be the number of orthonormal vectors returned by grahm_schmidt applied to the matrix [A b]
    # your code here
    throw(ErrorException())
    if nColsGS_A==nColsGS_Ab
        return true
    else
        return false
    end
end

In [ ]:
# Friendly self test
A1= [1 4 6; 2 3 3; 4 5 6]
b1= [1; 5; 4]
Exist1=sol_exist(A1, b1)
A2= [1 4; 2 3; 4 5]
b2= [1; 11; pi]
Exist2=sol_exist(A2, b2)
@show [Exist1 Exist2]
T1=@assert(Exist1==true)
T2=@assert(Exist2==false)
println("all nothings means likely correct")
[T1, T2]

### Problem 4. Another way to think about least squares to find solution to Ax=b when A is a tall matrix

We know from our work on regression and least squares solutions to linear equations, that if the columns of A are linearly independent, then the solution to $A^\top \cdot A x = A^\top b$ **exists and is uniuqe**. We can use this to determine if a solution to $Ax=b$ exists when $A$ is a tall matrix (more rows than columns).

**Fact:** Suppose the columns of A are linearly independent. Then 
$ Ax=b$ has a solution if, and only if, the unique solution to $A^\top \cdot A x^* = A^\top b$ satisfies $Ax^*-b=0$

Hence, our **pipeline** is: we **first solve the auxiliary problem** $A^\top \cdot A x^* = A^\top b$ and **then CHECK** if it also solves the original problem, $Ax=b$

In [ ]:
# We will walk through this example function together
function solveTallAxEqb(A,b,aTol=1e-8)
    # your code here
    throw(ErrorException())
end

In [ ]:
# Handy public autograder cell
A3= [1 4; 2 3; 4 5]
b3= pi*A3[:,1]-sqrt(17)*A3[:,2]
x3=solveTallAxEqb(A3,b3)
T1 = @assert norm(x3 - [3.141592653589792; -4.123105625617659]) < 1e-6
A4=A3
b4=[1;1;1]
x4=solveTallAxEqb(A4,b4)
T2 = @assert size(x4,1) == 0
println("all nothings means likely correct")
[T1 T2]

### Problem 5. Testing if $v \in V$ where $V = {\rm span}\{v_1, v_2, \cdots, v_k\}$ and we know that $ \{v_1, v_2, \cdots, v_k\}$ are linearly independent. 

We note that $v \in V \iff \text{we can find}~~~ \alpha_1, \alpha_2, \ldots, \alpha_k$ such that 
$v=\alpha_1 v_1 +  \alpha_2 v_2 + \cdots+  \alpha_k v_k$

Hence, we can define $A=\left[v_1 ~~~ v_2 ~~~ \cdots ~~~ v_k  \right]$, $\alpha=\begin{bmatrix} \alpha_1\\ \alpha_2\\ \vdots \\ \alpha_k \end{bmatrix} $ and check if $A \alpha = v$ has an exact solution or not! 

In [ ]:
# check whether the v and w are elements of span{v1, v2, v3, v4} where
Random.seed!(1984)
v1= randn(5,1)
v2= rand(5,1)
v3= rand(5,1)
v4= randn(5,1)

# and if so, find the coefficients such that v or w = a1 v1 + a2 v2 + a3 v3 + a4 v4

# Check separately
v= [ -1.2022572669353286
  0.3086799784045685
 -0.4601293761546652
 -0.25138545639836174
 -0.8894044975050598]
w= [ -0.7053977806057057
  0.5913041494333285
 -0.25576107276531757
 -1.1972897199751336
  1.0962030000908147]

## Your solution starts here
# your code here
throw(ErrorException())

We conclude that v is in span{v1, v2, v3, v4} and w is not! Moreover, we see that v = v1+0v2+v3+v4